In [1]:
from diffUV import dyned_eul, dyn_body, kin
from diffUV.utils.symbols import *
from casadi import Function
from blue_rov import Params as ps
import numpy as np

In [2]:
r_g = vertcat(x_g, y_g, z_g) # center of gravity wrt body origin
r_b = vertcat(x_b, y_b, z_b) # center of buoyancy wrt body origin 
I_o = vertcat(I_x, I_y, I_z,I_xz) # rigid body inertia wrt body origin

added_m = vertcat(X_du, Y_dv, Z_dw, K_dp, M_dq, N_dr) # added mass in diagonals
coupling_added_m =  vertcat(X_dq, Y_dp, N_dp, M_du, K_dv) # effective added mass in non diagonals

linear_dc = vertcat(X_u, Y_v, Z_w, K_p,  M_q, N_r) # linear damping coefficients
quadratic_dc = vertcat(X_uu, Y_vv, Z_ww, K_pp, M_qq, N_rr) # quadratic damping coefficients

n0 = vertcat(dx, dy, dz, thet, phi, psi, dphi, dthet, dpsi) # state variables wrt ned
v0 = x_nb # velocity state variables wrt body

<h1>Kinematics ned Transformation Matrix</h1>

In [3]:
Kinematics = kin()
J = Kinematics.J
J_inv = Kinematics.J_inv
print(J_inv)
Onb = vertcat(thet, phi, psi)
J_inv_func = Function('J_inv', [Onb], [J_inv]) # for numerical & symbolic use

@1=0, 
[[(cos(psi)*cos(phi)), (sin(psi)*cos(phi)), (-sin(phi)), @1, @1, @1], 
 [(((cos(psi)*sin(phi))*sin(thet))-(sin(psi)*cos(thet))), ((cos(psi)*cos(thet))+((sin(thet)*sin(phi))*sin(psi))), (cos(phi)*sin(thet)), @1, @1, @1], 
 [((sin(psi)*cos(thet))+((cos(psi)*cos(thet))*sin(phi))), (((sin(phi)*sin(psi))*sin(thet))-(cos(psi)*cos(thet))), (cos(phi)*cos(thet)), @1, @1, @1], 
 [@1, @1, @1, 1, 00, (-sin(phi))], 
 [@1, @1, @1, 00, cos(thet), (cos(phi)*sin(thet))], 
 [@1, @1, @1, 00, (-sin(thet)), (cos(phi)*cos(thet))]]


In [4]:
# example usage of J mat
jinv = np.array(J_inv_func([0.13,0.17,0.1]))
jinv

array([[ 0.98066095,  0.09839429, -0.16918235,  0.        ,  0.        ,
         0.        ],
       [-0.07716877,  0.98879774,  0.12776544,  0.        ,  0.        ,
         0.        ],
       [ 0.26590771, -0.98441869,  0.9772683 ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        -0.16918235],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.99156189,
         0.12776544],
       [ 0.        ,  0.        ,  0.        ,  0.        , -0.12963414,
         0.9772683 ]])

<h1>System Inertia Matrix example</h1>

In [5]:
# body representaion
uv_body = dyn_body()
inertia_mat = uv_body.body_inertia_matrix()
M_func = Function('M_b', [m, I_o, z_g, added_m, coupling_added_m], [inertia_mat]) # for numerical & symbolic use

In [6]:
# example usage of inertia mat
M_mat = np.array(M_func(ps.m, ps.Io, ps.rg[2], ps.added_m, ps.coupl_added_m))
M_mat

array([[17.  ,  0.  ,  0.  ,  0.  ,  0.23,  0.  ],
       [ 0.  , 24.2 ,  0.  , -0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  , 26.07,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.28,  0.  ,  0.  ],
       [ 0.23,  0.  ,  0.  ,  0.  ,  0.28,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.  ,  0.  ,  0.28]])

In [7]:
# ned representaion
uv_ned = dyned_eul()
inertia_ned_mat = uv_ned.ned_euler_inertia_matrix()
Onb = vertcat(thet, phi, psi)
M_ned_func = Function('M_ned', [m, I_o, z_g, Onb, added_m, coupling_added_m], [inertia_ned_mat]) # for numerical & symbolic use

In [11]:
# example usage of inertia mat in ned
Onb_v = np.array([0.13,0.17,0.1])
np.array(M_ned_func(ps.m, ps.Io, ps.rg[2], Onb_v,  ps.added_m, ps.coupl_added_m))

array([[ 1.83362708e+01, -7.03041061e+00,  3.71555407e+00,
         1.77488172e-02,  2.23648786e-01,  2.58149654e-02],
       [-7.03041061e+00,  4.90893516e+01, -2.23061182e+01,
        -2.27423481e-01,  2.24397276e-02,  4.13674584e-02],
       [ 3.71555407e+00, -2.23061182e+01,  2.57798665e+01,
        -2.93860503e-02, -3.85835972e-02, -8.67361738e-19],
       [ 1.77488172e-02, -2.27423481e-01, -2.93860503e-02,
         2.80000000e-01,  0.00000000e+00, -4.73710577e-02],
       [ 2.23648786e-01,  2.24397276e-02, -3.85835972e-02,
         0.00000000e+00,  2.80000000e-01,  0.00000000e+00],
       [ 2.58149654e-02,  4.13674584e-02,  8.67361738e-19,
        -4.73710577e-02,  0.00000000e+00,  2.80000000e-01]])

In [18]:
# checks --> above results (inertia mat in ned using body inertia and J tansform)
jinv.T@M_mat@jinv 

array([[ 1.83362708e+01, -7.03041061e+00,  3.71555407e+00,
         1.77488172e-02,  2.23648786e-01,  2.58149654e-02],
       [-7.03041061e+00,  4.90893516e+01, -2.23061182e+01,
        -2.27423481e-01,  2.24397276e-02,  4.13674584e-02],
       [ 3.71555407e+00, -2.23061182e+01,  2.57798665e+01,
        -2.93860503e-02, -3.85835972e-02, -7.71263355e-19],
       [ 1.77488172e-02, -2.27423481e-01, -2.93860503e-02,
         2.80000000e-01,  0.00000000e+00, -4.73710577e-02],
       [ 2.23648786e-01,  2.24397276e-02, -3.85835972e-02,
         0.00000000e+00,  2.80000000e-01, -2.51507399e-18],
       [ 2.58149654e-02,  4.13674584e-02,  7.71263355e-19,
        -4.73710577e-02, -2.45961687e-18,  2.80000000e-01]])

<h1>Forward dynamics simulation example</h1>

In [19]:
uv_ned = dyned_eul()
forward_dynamics = uv_ned.ned_euler_forward_dynamics()
# forward_dynamics.size()

fd_func = Function('f', [m, W, B, r_g, r_b, I_o, 
                   added_m, coupling_added_m, 
                   linear_dc, quadratic_dc,
                   v0, n0, tau_b], [forward_dynamics])

In [20]:
# example usage of forward dynamics
tau = np.array([1,0,0,0,0,0])
v0_v = np.array([0, 0, 0 ,0 ,0, 0])
n0_v = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
fd_func(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m, ps.coupl_added_m, ps.linear_dc,ps.quadratic_dc,v0_v,  n0_v,  tau)

DM([0.0594846, 4.16334e-17, -0.0767165, 2.34535e-15, -0.0488624, 0])